In [75]:
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools
from itertools import permutations,combinations
pd.set_option('max_row',None)
pd.set_option('max_columns',None)

## Markov chain

In [14]:
df = pd.read_csv('attribution data.csv')

In [15]:
df.head()

,cookie,time,interaction,conversion,conversion_value,channel
0,00000FkCnDfDDf0iC97iC703B,2018-07-03T13:02:11Z,impression,0,0.0,Instagram
1,00000FkCnDfDDf0iC97iC703B,2018-07-17T19:15:07Z,impression,0,0.0,Online Display
2,00000FkCnDfDDf0iC97iC703B,2018-07-24T15:51:46Z,impression,0,0.0,Online Display
3,00000FkCnDfDDf0iC97iC703B,2018-07-29T07:44:51Z,impression,0,0.0,Online Display
4,0000nACkD9nFkBBDECD3ki00E,2018-07-03T09:44:57Z,impression,0,0.0,Paid Search


In [54]:
def preprocess(df):
    df = df.sort_values(['cookie', 'time'],
                    ascending=[False, True])
    df['visit_order'] = df.groupby('cookie').cumcount() + 1
    df_paths = df.groupby('cookie')['channel'].aggregate(lambda x: x.tolist()).reset_index()
    df_last_interaction = df.drop_duplicates('cookie', keep='last')[['cookie', 'conversion']]
    df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='cookie')
    df_paths['path'] = df_paths['channel'].apply(lambda x: ">>>".join(x))
    return df_paths
    

In [16]:
df = df.sort_values(['cookie', 'time'],
                    ascending=[False, True])

df['visit_order'] = df.groupby('cookie').cumcount() + 1
df_paths = df.groupby('cookie')['channel'].aggregate(
    lambda x: x.tolist()).reset_index()

df_last_interaction = df.drop_duplicates('cookie', keep='last')[['cookie', 'conversion']]
df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='cookie')

#df_paths['conversion'] = df_paths['conversion'].map({0:'Null',1:'Conversion'})

In [17]:
df_paths.head()

,cookie,channel,conversion
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display, Online Display, On...",0
1,0000nACkD9nFkBBDECD3ki00E,"[Paid Search, Paid Search, Paid Search, Paid S...",0
2,0003EfE37E93D0BC03iBhBBhF,"[Paid Search, Paid Search, Paid Search, Paid S...",0
3,00073CFE3FoFCn70fBhB3kfon,[Instagram],0
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search],0


In [18]:
df_paths.conversion.value_counts()

0    222469
1     17639
Name: conversion, dtype: int64

In [19]:
df_paths['path'] = df_paths['channel'].apply(lambda x: ">>>".join(x))

In [20]:
df_paths.head()

,cookie,channel,conversion,path
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display, Online Display, On...",0,Instagram>>>Online Display>>>Online Display>>>...
1,0000nACkD9nFkBBDECD3ki00E,"[Paid Search, Paid Search, Paid Search, Paid S...",0,Paid Search>>>Paid Search>>>Paid Search>>>Paid...
2,0003EfE37E93D0BC03iBhBBhF,"[Paid Search, Paid Search, Paid Search, Paid S...",0,Paid Search>>>Paid Search>>>Paid Search>>>Paid...
3,00073CFE3FoFCn70fBhB3kfon,[Instagram],0,Instagram
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search],0,Paid Search


In [35]:
##现成的markov chain package
import pandas as pd
from pychattr.channel_attribution import MarkovModel

data = {
    "path": df_paths.path.values.tolist(),
    "conversion": df_paths.conversion.values.tolist() ,
#     "revenue": [0, 1, 0, 1, 1],
#     "cost": [0, 1, 0, 1, 1]
}

df = pd.DataFrame(data)

path_feature= 'path'
conversion_feature="conversion"
null_feature=None
#revenue_feature="revenue"
#cost_feature="cost"
separator=">>>"
k_order=1
n_simulations=10000
max_steps=None
return_transition_probs=True
random_state=26

# instantiate the model
mm = MarkovModel(path_feature=path_feature,
                 conversion_feature=conversion_feature,
                 null_feature=null_feature,
                 #revenue_feature=revenue_feature,
                 #cost_feature=cost_feature,
                 separator=separator,
                 k_order=k_order,
                 n_simulations=n_simulations,
                 max_steps=max_steps,
                 return_transition_probs=return_transition_probs,
                 random_state=random_state)

# fit the model
mm.fit(df)

In [36]:
df.head()

,path,conversion
0,Instagram>>>Online Display>>>Online Display>>>...,0
1,Paid Search>>>Paid Search>>>Paid Search>>>Paid...,0
2,Paid Search>>>Paid Search>>>Paid Search>>>Paid...,0
3,Instagram,0
4,Paid Search,0


In [38]:
df.conversion.value_counts()

0    222469
1     17639
Name: conversion, dtype: int64

In [39]:
# view the simulation results
print(mm.attribution_model_)

     channel_name  total_conversions
0       Instagram        3488.292950
1  Online Display        2007.900929
2     Paid Search        3967.540405
3        Facebook        5269.477284
4    Online Video        2905.788432


In [40]:
# view the transition matrix
print(mm.transition_matrix_)

      channel_from      channel_to  transition_probability
0          (start)     Paid Search                0.269686
1          (start)        Facebook                0.293497
2          (start)    Online Video                0.182323
3          (start)       Instagram                0.132037
4          (start)  Online Display                0.122456
5        Instagram    Online Video                0.027192
6        Instagram        Facebook                0.475483
7        Instagram    (conversion)                0.245058
8        Instagram       Instagram                0.204980
9        Instagram     Paid Search                0.030141
10       Instagram  Online Display                0.017145
11  Online Display    (conversion)                0.397584
12  Online Display  Online Display                0.393123
13  Online Display       Instagram                0.028067
14  Online Display     Paid Search                0.097212
15  Online Display    Online Video                0.0241

In [41]:
# view the removal effects
print(mm.removal_effects_)

     channel_name  removal_effect
0       Instagram          0.3108
1  Online Display          0.1789
2     Paid Search          0.3535
3        Facebook          0.4695
4    Online Video          0.2589


## Shapley value

In [47]:
raw = pd.read_csv('attribution data.csv')

In [48]:
raw.head()

,cookie,time,interaction,conversion,conversion_value,channel
0,00000FkCnDfDDf0iC97iC703B,2018-07-03T13:02:11Z,impression,0,0.0,Instagram
1,00000FkCnDfDDf0iC97iC703B,2018-07-17T19:15:07Z,impression,0,0.0,Online Display
2,00000FkCnDfDDf0iC97iC703B,2018-07-24T15:51:46Z,impression,0,0.0,Online Display
3,00000FkCnDfDDf0iC97iC703B,2018-07-29T07:44:51Z,impression,0,0.0,Online Display
4,0000nACkD9nFkBBDECD3ki00E,2018-07-03T09:44:57Z,impression,0,0.0,Paid Search


In [80]:
def preprocess_shapley(df):
    df = df.sort_values(['cookie', 'time'],
                    ascending=[False, True])
    df['visit_order'] = df.groupby('cookie').cumcount() + 1
    df_paths = df.groupby('cookie')['channel'].aggregate(lambda x: x.unique().tolist()).reset_index()
    df_last_interaction = df.drop_duplicates('cookie', keep='last')[['cookie', 'conversion']]
    df_paths = pd.merge(df_paths, df_last_interaction, how='left', on='cookie')
    df_paths['path'] = df_paths['channel'].apply(lambda x: ">>>".join(x))
    return df_paths

In [81]:
processed = preprocess_shapley(raw)

In [82]:
processed.head()

,cookie,channel,conversion,path
0,00000FkCnDfDDf0iC97iC703B,"[Instagram, Online Display]",0,Instagram>>>Online Display
1,0000nACkD9nFkBBDECD3ki00E,[Paid Search],0,Paid Search
2,0003EfE37E93D0BC03iBhBBhF,[Paid Search],0,Paid Search
3,00073CFE3FoFCn70fBhB3kfon,[Instagram],0,Instagram
4,00079hhBkDF3k3kDkiFi9EFAD,[Paid Search],0,Paid Search


In [83]:
processed_data = processed.groupby('path',as_index = False).agg({'conversion':'sum'})

In [84]:
processed_data.head()

,path,conversion
0,Facebook,3001
1,Facebook>>>Instagram,1345
2,Facebook>>>Instagram>>>Online Display,36
3,Facebook>>>Instagram>>>Online Display>>>Online...,2
4,Facebook>>>Instagram>>>Online Display>>>Online...,0


In [85]:
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in itertools.combinations(List, i+1)]
    return PS

In [86]:
def subsets(s):
    '''
    This function returns all the possible subsets of a set of channels.
    input :
            - s: a set of channels.
    '''
    if len(s)==1:
        return s
    else:
        sub_channels=[]
        for i in range(1,len(s)+1):
            sub_channels.extend(map(list,itertools.combinations(s, i)))
    return list(map(">>>".join,map(sorted,sub_channels)))

def v_function(A,C_values):
    '''
    This function computes the worth of each coalition.
    inputs:
            - A : a coalition of channels.
            - C_values : A dictionnary containing the number of conversions that each subset of channels has yielded.
    '''
    subsets_of_A = subsets(A)
    #print(subsets_of_A)
    #exit()
    worth_of_A=0
    for subset in subsets_of_A:
        #print("subset:", subset)
        if subset in C_values:
            #print("subset:", subset, "; Value:", C_values[subset])
            worth_of_A += C_values[subset]
    return worth_of_A

def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


def calculate_shapley(df, col_name):
    '''
    This function returns the shapley values
            - df: A dataframe with the two columns: ['path', 'conversion'].
            The channel_subset column is the channel(s) associated with the conversion and the count is the sum of the conversions. 
            - col_name: A string that is the name of the column with conversions          
    '''
    c_values = df.set_index("path").to_dict()[col_name]
    df['channels'] = df['path'].apply(lambda x: x if len(x.split(">>>")) == 1 else np.nan)
    channels = list(df['channels'].dropna().unique())
    
    v_values = {}
    for A in power_set(channels):
        v_values['>>>'.join(sorted(A))] = v_function(A,c_values)
    #print(v_values)
    n=len(channels)
    shapley_values = defaultdict(int)
    for channel in channels:
        for A in v_values.keys():
            #print(A)
            if channel not in A.split(">>>"):
                #print(channel)
                cardinal_A=len(A.split(">>>"))
                A_with_channel = A.split(">>>")
                A_with_channel.append(channel)            
                A_with_channel=">>>".join(sorted(A_with_channel))
                # Weight = |S|!(n-|S|-1)!/n!
                weight = (factorial(cardinal_A)*factorial(n-cardinal_A-1)/factorial(n))
                # Marginal contribution = v(S U {i})-v(S)
                contrib = (v_values[A_with_channel]-v_values[A]) 
                shapley_values[channel] += weight * contrib
        # Add the term corresponding to the empty set
        shapley_values[channel]+= v_values[channel]/n 

    return shapley_values

In [87]:
shapley_values = calculate_shapley(processed_data,'conversion')

In [88]:
shapley_values

defaultdict(int,
            {'Facebook': 3956.916666666667,
             'Instagram': 1834.0,
             'Online Display': 1811.5833333333335,
             'Online Video': 2854.916666666667,
             'Paid Search': 4144.583333333333})